In [23]:
import os

In [24]:
from beir import util, LoggingHandler
from beir.datasets.data_loader import GenericDataLoader
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval.search.lexical import BM25Search as BM25


In [25]:
import logging

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])

In [26]:
dataset = "scifact"
url = "https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{}.zip".format(dataset)
out_dir = "./datasets"
data_path = util.download_and_unzip(url, out_dir)

In [27]:
corpus, queries, qrels = GenericDataLoader(data_path).load(split="test")# pull data from corpus and queries

2024-01-24 06:41:45 - Loading Corpus...


100%|██████████| 5183/5183 [00:00<00:00, 71897.76it/s]

2024-01-24 06:41:45 - Loaded 5183 TEST Documents.
2024-01-24 06:41:45 - Doc Example: {'text': 'Alterations of the architecture of cerebral white matter in the developing human brain can affect cortical development and result in functional disabilities. A line scan diffusion-weighted magnetic resonance imaging (MRI) sequence with diffusion tensor analysis was applied to measure the apparent diffusion coefficient, to calculate relative anisotropy, and to delineate three-dimensional fiber architecture in cerebral white matter in preterm (n = 17) and full-term infants (n = 7). To assess effects of prematurity on cerebral white matter development, early gestation preterm infants (n = 10) were studied a second time at term. In the central white matter the mean apparent diffusion coefficient at 28 wk was high, 1.8 microm2/ms, and decreased toward term to 1.2 microm2/ms. In the posterior limb of the internal capsule, the mean apparent diffusion coefficients at both times were similar (1.2 vers

In [28]:
# next(iter(corpus)), next(iter(corpus.items()))

In [29]:
# for id in corpus:
#     print("corpus_id:", id)
#     print("title:", corpus[id]["title"]) 
#     print("text:", corpus[id]["text"][:100], "...")


In [30]:
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchIndex,
    ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField
)

service_name = "fta-ai-search"
admin_key = "UDVpljtfzJwteq82dTh9Iu68CMtHDEVdTNyEmkoPvDAzSeC0EAMU"

index_name = "scifact-tmp"

# Create an SDK client
endpoint = "https://{}.search.windows.net/".format(service_name)
admin_client = SearchIndexClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

try:
    result = admin_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

2024-01-24 06:41:45 - Request URL: 'https://fta-ai-search.search.windows.net/indexes('scifact-tmp')?api-version=REDACTED'
Request method: 'DELETE'
Request headers:
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=minimal'
    'x-ms-client-request-id': 'a45656ee-ba83-11ee-b15e-0022485a4ca6'
    'User-Agent': 'azsdk-python-search-documents/11.4.0 Python/3.8.18 (Linux-6.2.0-1018-azure-x86_64-with-glibc2.17)'
No body was attached to the request
2024-01-24 06:41:45 - Response status: 404
Response headers:
    'Cache-Control': 'no-cache,no-store'
    'Pragma': 'no-cache'
    'Content-Length': '276'
    'Content-Type': 'application/json; charset=utf-8'
    'Content-Language': 'REDACTED'
    'Expires': '-1'
    'Server': 'Microsoft-IIS/10.0'
    'request-id': 'a45656ee-ba83-11ee-b15e-0022485a4ca6'
    'elapsed-time': 'REDACTED'
    'Strict-Transport-Security': 'REDACTED'
    'Date': 'Wed, 24 Jan 2024 06:41:45 GMT'
Index scifact-tmp Deleted


In [31]:
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchableField,
    SimpleField,
    ScoringProfile,
    SearchIndex,
    SearchFieldDataType
)


fields = [
    SimpleField(name="corpusId", type=SearchFieldDataType.String, key=True),
    SearchableField(name="title", type=SearchFieldDataType.String),
    SearchableField(name="text", type=SearchFieldDataType.String),
]
cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)
scoring_profiles = []
index = SearchIndex(
    name=index_name,
    fields=fields,
    scoring_profiles=scoring_profiles,
    cors_options=cors_options)


try:
    result = admin_client.create_index(index)
    print ('Index', result.name, 'created')
except Exception as ex:
    print (ex)

2024-01-24 06:41:46 - Request URL: 'https://fta-ai-search.search.windows.net/indexes?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '593'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=minimal'
    'x-ms-client-request-id': 'a499f53e-ba83-11ee-b15e-0022485a4ca6'
    'User-Agent': 'azsdk-python-search-documents/11.4.0 Python/3.8.18 (Linux-6.2.0-1018-azure-x86_64-with-glibc2.17)'
A body is sent with the request
2024-01-24 06:41:46 - Response status: 201
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=minimal; odata.streaming=true; charset=utf-8'
    'ETag': '"0x8DC1CA7891CCCEF"'
    'Location': 'REDACTED'
    'Server': 'Microsoft-IIS/10.0'
    'Strict-Transport-Security': 'REDACTED'
    'Preference-Applied': 'REDACTED'
    'OData-Version': 'REDACTED'
    'request-id': 'a499f53e-ba83-11ee-b15e-0022485a4ca6'
    'elapsed-time': 'RE

In [32]:
# create documents for corpus
documents = []
for id in corpus:
    #print(id)
    documents.append({
        "corpusId": id,
        "title": corpus[id]["title"],
        "text": corpus[id]["text"]
    })

In [33]:
try:
    result = search_client.upload_documents(documents=documents)
    print("Upload of new document succeeded: {}".format(result[0].succeeded))
except Exception as ex:
    print (ex.message)

2024-01-24 06:41:46 - Request URL: 'https://fta-ai-search.search.windows.net//indexes('scifact-tmp')/docs/search.index?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '8199871'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': 'a50b568e-ba83-11ee-b15e-0022485a4ca6'
    'User-Agent': 'azsdk-python-search-documents/11.4.0 Python/3.8.18 (Linux-6.2.0-1018-azure-x86_64-with-glibc2.17)'
A body is sent with the request
2024-01-24 06:41:49 - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streaming=true; charset=utf-8'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Server': 'Microsoft-IIS/10.0'
    'Strict-Transport-Security': 'REDACTED'
    'Preference-Applied': 'REDACTED'
    'OData-Version': 'REDACTED'
    'request-id': 'a50b568e-ba83-11ee-b15e-0022485a

## Search an index

In [35]:
results = search_client.search(search_text="*", include_total_count=True)

print ('Total Documents Matching Query:', results.get_count())
# for result in results:
#     print(result)

2024-01-24 06:43:58 - Request URL: 'https://fta-ai-search.search.windows.net//indexes('scifact-tmp')/docs/search.post.search?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '30'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': 'f3c7bbaa-ba83-11ee-b15e-0022485a4ca6'
    'User-Agent': 'azsdk-python-search-documents/11.4.0 Python/3.8.18 (Linux-6.2.0-1018-azure-x86_64-with-glibc2.17)'
A body is sent with the request
2024-01-24 06:43:58 - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streaming=true; charset=utf-8'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Server': 'Microsoft-IIS/10.0'
    'Strict-Transport-Security': 'REDACTED'
    'Preference-Applied': 'REDACTED'
    'OData-Version': 'REDACTED'
    'request-id': 'f3c7bbaa-ba83-11ee-b15e-0022485

In [47]:
results = search_client.search(search_text="Micro", include_total_count=True, select='corpusId, title, text')

print ('Total Documents Matching Query:', results.get_count())
for result in results:
    print("corpusId:", result["corpusId"])
    print("title", result["title"])
    print("text", result["text"][:100], "...\n")

2024-01-24 06:50:57 - Request URL: 'https://fta-ai-search.search.windows.net//indexes('scifact-tmp')/docs/search.post.search?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '69'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': 'ed247e4a-ba84-11ee-b15e-0022485a4ca6'
    'User-Agent': 'azsdk-python-search-documents/11.4.0 Python/3.8.18 (Linux-6.2.0-1018-azure-x86_64-with-glibc2.17)'
A body is sent with the request
2024-01-24 06:50:57 - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streaming=true; charset=utf-8'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Server': 'Microsoft-IIS/10.0'
    'Strict-Transport-Security': 'REDACTED'
    'Preference-Applied': 'REDACTED'
    'OData-Version': 'REDACTED'
    'request-id': 'ed247e4a-ba84-11ee-b15e-0022485

In [116]:
query_ids = list(queries)
dict_results = {}
for query_id in query_ids:
    query = queries[query_id]
    results = search_client.search(search_text=query, include_total_count=True, select='corpusId, title, text', top=5)
    # dict_results = (dict(result) for result in results)
    id_score = {}
    for result in results:
        id_score[result["corpusId"]] = result["@search.score"]
    # print(id_score)
    dict_results[query_id] = id_score
    # print(dict_results)

2024-01-24 07:55:22 - Request URL: 'https://fta-ai-search.search.windows.net//indexes('scifact-tmp')/docs/search.post.search?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '127'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': 'ecd0b388-ba8d-11ee-b15e-0022485a4ca6'
    'User-Agent': 'azsdk-python-search-documents/11.4.0 Python/3.8.18 (Linux-6.2.0-1018-azure-x86_64-with-glibc2.17)'
A body is sent with the request
2024-01-24 07:55:22 - Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streaming=true; charset=utf-8'
    'Content-Encoding': 'REDACTED'
    'Vary': 'REDACTED'
    'Server': 'Microsoft-IIS/10.0'
    'Strict-Transport-Security': 'REDACTED'
    'Preference-Applied': 'REDACTED'
    'OData-Version': 'REDACTED'
    'request-id': 'ecd0b388-ba8d-11ee-b15e-002248

In [117]:
dict_results.keys()

dict_keys(['1', '3', '5', '13', '36', '42', '48', '49', '50', '51', '53', '54', '56', '57', '70', '72', '75', '94', '99', '100', '113', '115', '118', '124', '127', '128', '129', '130', '132', '133', '137', '141', '142', '143', '146', '148', '163', '171', '179', '180', '183', '185', '198', '208', '212', '213', '216', '217', '218', '219', '230', '232', '233', '236', '237', '238', '239', '248', '249', '261', '268', '269', '274', '275', '279', '294', '295', '298', '300', '303', '312', '314', '324', '327', '338', '343', '350', '354', '362', '380', '384', '385', '386', '388', '399', '410', '411', '415', '421', '431', '436', '437', '439', '440', '443', '452', '475', '478', '491', '501', '502', '507', '508', '513', '514', '516', '517', '521', '525', '527', '528', '532', '533', '535', '536', '539', '540', '544', '549', '551', '552', '554', '560', '569', '575', '577', '578', '587', '589', '593', '597', '598', '613', '619', '623', '628', '636', '637', '641', '644', '649', '659', '660', '674', '68

In [115]:
next(iter(dict_results.items()))

('1',
 {'43385013': 13.602742,
  '10608397': 13.154596,
  '10029970': 11.521676,
  '121581019': 11.402816,
  '42240424': 11.372993})

In [119]:
from beir.retrieval.evaluation import EvaluateRetrieval
ndcg, _map, recall, precision = EvaluateRetrieval.evaluate(qrels, dict_results, [1, 3, 5, 10, 100, 1000])
print(ndcg, _map, recall, precision)


2024-01-24 08:04:49 - For evaluation, we ignore identical query and document ids (default), please explicitly set ``ignore_identical_ids=False`` to ignore this.
2024-01-24 08:04:49 - 

2024-01-24 08:04:49 - NDCG@1: 0.5200
2024-01-24 08:04:49 - NDCG@3: 0.5840
2024-01-24 08:04:49 - NDCG@5: 0.6110
2024-01-24 08:04:49 - NDCG@10: 0.6110
2024-01-24 08:04:49 - NDCG@100: 0.6110
2024-01-24 08:04:49 - NDCG@1000: 0.6110
2024-01-24 08:04:49 - 

2024-01-24 08:04:49 - MAP@1: 0.4993
2024-01-24 08:04:49 - MAP@3: 0.5606
2024-01-24 08:04:49 - MAP@5: 0.5773
2024-01-24 08:04:49 - MAP@10: 0.5773
2024-01-24 08:04:49 - MAP@100: 0.5773
2024-01-24 08:04:49 - MAP@1000: 0.5773
2024-01-24 08:04:49 - 

2024-01-24 08:04:49 - Recall@1: 0.4993
2024-01-24 08:04:49 - Recall@3: 0.6307
2024-01-24 08:04:49 - Recall@5: 0.6937
2024-01-24 08:04:49 - Recall@10: 0.6937
2024-01-24 08:04:49 - Recall@100: 0.6937
2024-01-24 08:04:49 - Recall@1000: 0.6937
2024-01-24 08:04:49 - 

2024-01-24 08:04:49 - P@1: 0.5200
2024-01-24 08:04:49

In [ ]:
# ndcg, _map, recall, precision = retriever.evaluate(qrels, results, retriever.k_values)a